显示了可变序列（MutableSequence） 和不可变序列
（Sequence） 的差异， 同时也能看出前者从后者那里继承了一些方
法。 虽然内置的序列类型并不是直接从 Sequence 和
MutableSequence 这两个抽象基类（Abstract Base Class， ABC） 继承
而来的， 但是了解这些基类可以帮助我们总结出那些完整的序列类型包
含了哪些功能

<img src="img/MutableSequence.png">

通过记住这些类的共有特性， 把可变与不可变序列或是容器与扁平序列
的概念融会贯通， 在探索并学习新的序列类型时， 你会更加得心应手

列表推导也可能被滥用。 以前看到过有的 Python 代码用列表
推导来重复获取一个函数的副作用。 通常的原则是， 只用列表推导来创
建新的列表， 并且尽量保持简短。 如果列表推导的代码超过了两行， 你
可能就要考虑是不是得用 for 循环重写了。 就跟写文章一样， 并没有什
么硬性的规则， 这个度得你自己把握。

Python 会忽略代码里 []、 {} 和 () 中的换行， 因此如果你的代码里
有多行的列表、 列表推导、 生成器表达式、 字典这一类的， 可以省
略不太好看的续行符 \。

Python 2.x 中， 在列表推导中 for 关键词之后的赋值操作可能会影
响列表推导上下文中的同名变量。 像下面这个 Python 2.7 控制台对
话：

<img src="img/python2.png">

如你所见， x 原本的值被取代了， 但是这种情况在 Python 3 中是不
会出现的。列表推导、 生成器表达式， 以及同它们很相似的集合（set） 推导
和字典（dict） 推导， 在 Python 3 中都有了自己的局部作用域， 就
像函数似的。 表达式内部的变量和赋值只在局部起作用， 表达式的
上下文里的同名变量还可以被正常引用， 局部变量并不会影响到它
们。

In [3]:
x = 'ABC'
dummy = [ord(x) for x in x]
x

'ABC'

用列表推导和 map/filter 组合来创建同样的表单

In [4]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [5]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

我原以为 map/filter 组合起来用要比列表推导快一些， Alex Martelli
却说不一定——至少在上面这个例子中不一定。 在本书的代码仓库
（https://github.com/fluentpython/example-code） 中有名为 02-arrayseq/listcomp_speed.py（https://github.com/fluentpython/examplecode/blob/master/02-array-seq/listcomp_speed.py） 的脚本， 代码中有这两
个方法的效率的比较。

### 元组不仅仅是不可变的列表

有些 Python 入门教程把元组称为“不可变列表”， 然而这并没有完全概括
元组的特点。 除了用作不可变的列表， 它还可以用于没有字段名的记
录。 鉴于后者常常被忽略， 我们先来看看元组作为记录的功用。

元组其实是对数据的记录： 元组中的每个元素都存放了记录中一个字段
的数据， 外加这个字段的位置。 正是这个位置信息给数据赋予了意义。
如果只把元组理解为不可变的列表， 那其他信息——它所含有的元素的
总数和它们的位置——似乎就变得可有可无。 但是如果把元组当作一些
字段的集合， 那么数量和位置信息就变得非常重要了。
元组就被当作记录加以利用。 如果在任何的表达式里我们
在元组内对元素排序， 这些元素所携带的信息就会丢失， 因为这些信息
是跟它们的位置有关的。

In [6]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'),('ESP', 'XDA205856')]

In [9]:
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


展示了如何用具名元组来记录一个城市的信息。

In [10]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

除了从普通元组那里继承来的属性之外， 具名元组还有一些自己专有的
属性。 示例 2-10 中就展示了几个最有用的： _fields 类属性、 类方法
_make(iterable) 和实例方法 _asdict()。

In [11]:
City._fields

('name', 'country', 'population', 'coordinates')

In [12]:
LatLong = namedtuple('LatLong', 'lat long')

In [13]:
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))

In [14]:
delhi = City._make(delhi_data)

In [15]:
delhi._asdict()

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', LatLong(lat=28.613889, long=77.208889))])

### 切片

在 Python 里， 像列表（list） 、 元组（tuple） 和字符串（str） 这类
序列类型都支持切片操作， 但是实际上切片操作比人们所想象的要强大
很多。

In [2]:
my_list = [[1,2]] * 3
my_list

[[1, 2], [1, 2], [1, 2]]

如果在 a * n 这个语句中， 序列 a 里的元素是对其他可变
对象的引用的话， 你就需要格外注意了， 因为这个式子的结果可能
会出乎意料。 比如， 你想用 my_list = [[]] * 3 来初始化一个
由列表组成的列表， 但是你得到的列表里包含的 3 个元素其实是 3
个引用， 而且这 3 个引用指向的都是同一个列表。 这可能不是你想
要的效果。

含有 3 个指向同一对象的引用的列表是毫无用处的

In [4]:
my_list[1][1] = 0
my_list

[[1, 0], [1, 0], [1, 0]]

外面的列表其实包含 3 个指向同一个列表的引用。 当我们不做修改
的时候， 看起来都还好。
一旦我们试图标记第 1 行第 2 列的元素， 就立马暴露了列表内的 3
个引用指向同一个对象的事实。

有时我们会需要初始化一个嵌套着几个列表的列表， 譬如一个列表可能
需要用来存放不同的学生名单， 或者是一个井字游戏板 上的一行方
块。 想要达成这些目的， 最好的选择是使用列表推导

In [6]:
my_list1 = [[1,2] for i in range(3)]
my_list1

[[1, 2], [1, 2], [1, 2]]

In [7]:
my_list1[1][1] = 0
my_list1

[[1, 2], [1, 0], [1, 2]]

下面的两个表达式到底
会产生什么结果？ 

In [9]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [10]:
t

(1, 2, [30, 40, 50, 60])

Python Tutor（http://www.pythontutor.com） 是一个对 Python 运行原理进行
可视化分析的工具。 图 2-3 里是两张截图， 分别代表示例 2-15 中 t 的
初始和最终状态。

Python Tutor（http://www.pythontutor.com） 是一个对 Python 运行原理进行
可视化分析的工具。 图 2-3 里是两张截图， 分别代表示例 2-15 中 t 的
初始和最终状态。

这其实是个非常罕见的边界情况， 在 15 年的 Python 生涯中， 我还没见
过谁在这个地方吃过亏。
至此我得到了 3 个教训。
不要把可变对象放在元组里面。
增量赋值不是一个原子操作。 我们刚才也看到了， 它虽然抛出了异
常， 但还是完成了操作。
查看 Python 的字节码并不难， 而且它对我们了解代码背后的运行机
制很有帮助。
在见证了 + 和 * 的微妙之处后， 我们把话题转移到序列类型的另一个重
要部分上： 排序

### list.sort方法和内置函数sorted

list.sort 方法会就地排序列表， 也就是说不会把原列表复制一份。 这
也是这个方法的返回值是 None 的原因， 提醒你本方法不会新建一个列
表。 在这种情况下返回 None 其实是 Python 的一个惯例： 如果一个函数
或者方法对对象进行的是就地改动， 那它就应该返回 None， 好让调用
者知道传入的参数发生了变动， 而且并未产生新的对象。 例
如， random.shuffle 函数也遵守了这个惯例。

与 list.sort 相反的是内置函数 sorted， 它会新建一个列表作为返回
值。 这个方法可以接受任何形式的可迭代对象作为参数， 甚至包括不可
变序列或生成器（见第 14 章） 。 而不管 sorted 接受的是怎样的参
数， 它最后都会返回一个列表

不管是 list.sort 方法还是 sorted 函数， 都有两个可选的关键字参
数。  
reverse  
如果被设定为 True， 被排序的序列里的元素会以降序输出（也就
是说把最大值当作最小值来排序） 。 这个参数的默认值是 False。  
key  
一个只有一个参数的函数， 这个函数会被用在序列里的每一个元素  
上， 所产生的结果将是排序算法依赖的对比关键字。 比如说， 在对一些
字符串排序时， 可以用 key=str.lower 来实现忽略大小写的排序， 或
者是用 key=len 进行基于字符串长度的排序。 这个参数的默认值是恒

In [12]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

['apple', 'banana', 'grape', 'raspberry']

In [13]:
sorted(fruits, key=len)

['grape', 'apple', 'banana', 'raspberry']

In [14]:
fruits.sort()

In [15]:
fruits

['apple', 'banana', 'grape', 'raspberry']

### 用bisect来管理已排序的序列

bisect 模块包含两个主要函数， bisect 和 insort， 两个函数都利用
二分查找算法来在有序序列中查找或插入元素。

bisect(haystack, needle) 在 haystack（干草垛） 里搜索
needle（针） 的位置， 该位置满足的条件是， 把 needle 插入这个位置
之后， haystack 还能保持升序。 也就是在说这个函数返回的位置前面
的值， 都小于或等于 needle 的值。 其中 haystack 必须是一个有序的
序列。 你可以先用 bisect(haystack, needle) 查找位置 index， 再
用 haystack.insert(index, needle) 来插入新值。 但你也可用
insort 来一步到位， 并且后者的速度更快一些。

In [30]:
import bisect
import sys
HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]
ROW_FMT = '{0:2d} @ {1:2d} {2}{0:2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle) 
        offset = position * ' |' 
        print(ROW_FMT.format(needle, position, offset)) 
    
if sys.argv[-1] == 'left': 
    bisect_fn = bisect.bisect_left
else:
    bisect_fn = bisect.bisect
print('DEMO:', bisect_fn.__name__) 
print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
demo(bisect_fn)    

DEMO: bisect
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14  | | | | | | | | | | | | | |31
30 @ 14  | | | | | | | | | | | | | |30
29 @ 13  | | | | | | | | | | | | |29
23 @ 11  | | | | | | | | | | |23
22 @  9  | | | | | | | | |22
10 @  5  | | | | |10
 8 @  5  | | | | | 8
 5 @  3  | | | 5
 2 @  1  | 2
 1 @  1  | 1
 0 @  0  0


### 根据一个分数， 找到它所对应的成绩

In [32]:
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]
#小于60，i=0
[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

### 用bisect.insort插入新元素     
排序很耗时， 因此在得到一个有序序列之后， 我们最好能够保持它的有
序。 bisect.insort 就是为了这个而存在的。
insort(seq, item) 把变量 item 插入到序列 seq 中， 并能保持 seq
的升序顺序。 详见示例 2-19 和它在图 2-6 里的输出。

In [37]:
import bisect
import random
SIZE=7

random.seed(1729)
my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


### 当列表不是首选时

虽然列表既灵活又简单， 但面对各类需求时， 我们可能会有更好的选
择。 比如， 要存放 1000 万个浮点数的话， 数组（array） 的效率要高
得多， 因为数组在背后存的并不是 float 对象， 而是数字的机器翻
译， 也就是字节表述。 这一点就跟 C 语言中的数组一样。 再比如说， 如
果需要频繁对序列做先进先出的操作， deque（双端队列） 的速度应该
会更快。

#### 数组

如果我们需要一个只包含数字的列表， 那么 array.array 比 list 更
高效。 数组支持所有跟可变序列有关的操作， 包括 .pop、 .insert 和
.extend。 另外， 数组还提供从文件读取和存入文件的更快的方法， 如
.frombytes 和 .tofile。    
Python 数组跟 C 语言数组一样精简。 创建数组需要一个类型码， 这个类
型码用来表示在底层的 C 语言应该存放怎样的数据类型。 比如 b 类型码
代表的是有符号的字符（signed char） ， 因此 array('b') 创建出的
数组就只能存放一个字节大小的整数， 范围从 -128 到 127， 这样在序列
很大的时候， 我们能节省很多空间。 而且 Python 不会允许你在数组里存
放除指定类型之外的数据。    
示例 2-20 展示了从创建一个有 1000 万个随机浮点数的数组开始， 到如
何把这个数组存放到文件里， 再到如何从文件读取这个数组。  

In [38]:
from array import array
from random import random
floats = array('d', (random() for i in range(10**7)))
floats[-1]

0.5963321947530882

从 Python 3.4 开始， 数组类型不再支持诸如 list.sort() 这种就地
排序方法。 要给数组排序的话， 得用 sorted 函数新建一个数组：

a = array.array(a.typecode, sorted(a))

### 内存视图

如果你总是跟数组打交道， 却没有听过 memoryview， 那就太遗憾了。
下面就来谈谈 memoryview。

memoryview 是一个内置类， 它能让用户在不复制内容的情况下操作同
一个数组的不同切片。 memoryview 的概念受到了 NumPy 的启发（参见
2.9.3 节） 。 Travis Oliphant 是 NumPy 的主要作者， 他在回答“ When
should a memoryview be
used?”（http://stackoverflow.com/questions/4845418/when-should-amemoryview-be-used/） 这个问题时是这样说的：
内存视图其实是泛化和去数学化的 NumPy 数组。 它让你在不需要
复制内容的前提下， 在数据结构之间共享内存。 其中数据结构可以
是任何形式， 比如 PIL图片、 SQLite 数据库和 NumPy 的数组， 等
等。 这个功能在处理大型数据集合的时候非常重要。      
memoryview.cast 的概念跟数组模块类似， 能用不同的方式读写同一
块内存数据， 而且内容字节不会随意移动。 这听上去又跟 C 语言中类型
转换的概念差不多。 memoryview.cast 会把同一块内存里的内容打包成一个全新的 memoryview 对象给你。